In [3]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [4]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.exceptions import NotFittedError
from sklearn.preprocessing import StandardScaler
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.exceptions import UnknownLevelWarning
from skforecast.recursive import ForecasterRecursiveMultiSeries


series = pd.DataFrame({'l1': pd.Series(np.arange(10)), 
                       'l2': pd.Series(np.arange(10))})

In [11]:
forecaster = ForecasterRecursiveMultiSeries(
                LinearRegression(),
                lags=3,
                encoding='ordinal',
                )
forecaster.fit(series=series)
y_true = {'l1': np.array([1, 2, 3, 4, 5]), 'l2': np.array([2, 3, 4, 5, 6])}
y_pred = {'l1': np.array([0, 1, 2, 3, 4]), 'l2': np.array([0, 1, 2, 3, 4])}

forecaster.set_out_sample_residuals(y_true=y_true, y_pred=y_pred)
results = forecaster.out_sample_residuals_
expected = {
    'l1': np.array([1, 1, 1, 1, 1]),
    'l2': np.array([2, 2, 2, 2, 2]),
    '_unknown_level': np.array([2, 2, 2, 2, 2, 1, 1, 1, 1, 1])
}

assert expected.keys() == results.keys()
assert all(all(np.sort(expected[k]) == np.sort(results[k])) for k in expected.keys())

results

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multiseries.py:436: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


{'l1': array([1, 1, 1, 1, 1]),
 'l2': array([2, 2, 2, 2, 2]),
 '_unknown_level': array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2])}

In [10]:
results

{'l1': array([1, 1, 1, 1, 1]),
 'l2': array([2, 2, 2, 2, 2]),
 '_unknown_level': array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2])}

In [43]:
[1, 2, 3, 4, 5] * 2

[1, 2, 3, 4, 5, 1, 2, 3, 4, 5]

In [43]:
forecaster.predict(steps = 10)

2024-01-01    116.765062
2024-01-02    123.313042
2024-01-03    126.326493
2024-01-04    128.998787
2024-01-05    132.018021
2024-01-06    134.986518
2024-01-07    137.623172
2024-01-08    139.927627
2024-01-09    141.982424
2024-01-10    143.839301
Freq: D, Name: pred, dtype: float64

In [44]:
# Backtesting predictions
# ==============================================================================
metric, predictions = backtesting_forecaster(
    forecaster            = forecaster,
    y                     = df_test["y"],
    steps                 = 1,
    metric                = 'mean_absolute_error',
    initial_train_size    = len(df_test) - 30,
    interval              = [2.5, 97.5],
    refit                 = True,
    n_jobs                = 1,
    verbose               = True,
    show_progress         = True,
    random_state          = 12345
)


Information of backtesting process
----------------------------------
Number of observations used for initial training: 3958
Number of observations used for backtesting: 30
    Number of folds: 30
    Number skipped folds: 0 
    Number of steps per fold: 1
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2013-01-30 00:00:00 -- 2023-12-01 00:00:00  (n=3958)
    Validation: 2023-12-02 00:00:00 -- 2023-12-02 00:00:00  (n=1)
Fold: 1
    Training:   2013-01-31 00:00:00 -- 2023-12-02 00:00:00  (n=3958)
    Validation: 2023-12-03 00:00:00 -- 2023-12-03 00:00:00  (n=1)
Fold: 2
    Training:   2013-02-01 00:00:00 -- 2023-12-03 00:00:00  (n=3958)
    Validation: 2023-12-04 00:00:00 -- 2023-12-04 00:00:00  (n=1)
Fold: 3
    Training:   2013-02-02 00:00:00 -- 2023-12-04 00:00:00  (n=3958)
    Validation: 2023-12-05 00:00:00 -- 2023-12-05 00:00:00  (n=1)
Fold: 4
    Training:   2013-02-03 00:00:00 -- 2023-12-05 00:00:00  (n=3958)
    Valid

  0%|          | 0/30 [00:00<?, ?it/s]

In [45]:
transformer_y    = FunctionTransformer(func=np.log1p, inverse_func=np.expm1, validate=True)
transformer_y.fit_transform(df_test[["y"]])

/home/ubuntu/anaconda3/envs/skforecast_13_py12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but FunctionTransformer was fitted with feature names
  warnings.warn(


array([[5.07196368],
       [5.07196368],
       [5.07196368],
       ...,
       [4.77267769],
       [4.55639565],
       [4.6532061 ]])

In [38]:
transformer_y    = StandardScaler()
transformer_y.fit_transform(df_test["y"].values.reshape(-1, 1))

array([[-0.00837553],
       [-0.00837553],
       [-0.00837553],
       ...,
       [-0.16196198],
       [-0.24757884],
       [-0.21153412]])

In [ ]:

forecaster = ForecasterAutoreg(
    regressor        = LinearRegression(),
    lags             = 3, # using past 3 days
    #transformer_y    = transformer_y,  <============
    transformer_exog = transformer_exog
)